In [38]:
import pandas as pd
import pickle
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support
from preprocessing import PreProcessor
preprocess = PreProcessor().preprocess
from sklearn.model_selection import train_test_split
import warnings
import numpy as np
warnings.simplefilter('ignore')

In [34]:
df_agora = pd.read_csv('data/Mapped.csv')
df_agora = df_agora.dropna(subset = ['Category', 'Item', 'Item Description'])
corpus_agora = df_agora['Item'] + df_agora['Item Description']
corpus_agora = corpus_agora.apply(lambda d: preprocess(d , lower_case=False, strip=False, punctuation=False, numbers=False, unicode=False, cut_off=False, stop_words=False, stemming=False, lemmatizing=False, min_word_length=-1, max_word_length=-1, tokenize=False))

In [35]:
tfidfAgora = TfidfVectorizer(ngram_range=(1,2))
X_Agora = tfidfAgora.fit_transform(corpus_agora)
Y_Agora = df_agora["Category"]

In [36]:
def QuickMap(sentence):
    if sentence == 'Drugs & Chemicals': return 'Drugs'
    if sentence == 'Guides & Tutorials': return 'Tutorial'
    if sentence == 'Software & Malware': return 'Software'
    if sentence == 'Counterfeits': return 'Counterfeit'
    if sentence == 'Jewelry & Gold': return 'Stolen Goods'
    if sentence == 'Security & Hosting': return 'Hosting'
    if sentence == 'Digital Goods': return 'Wiki'
    if sentence == 'Carded Items': return 'Carding'
    else: return sentence

with open('data/pickle_bu/tfidf_model.pkl', 'rb') as f:
    tfidfWebIQ = pickle.load(f)
with open('data/pickle_bu/tfidf_vectors.pkl', 'rb') as f:
    X_WebIQ = pickle.load(f)
with open('data/pickle_bu/categorieen.pkl', 'rb') as f:
    Y_WebIQ = pickle.load(f)
    
Y_WebIQ = pd.Series(Y_WebIQ).apply(lambda d: QuickMap(d))

In [37]:
print('Training for Agora')
X_A_train, X_A_test, Y_A_train, Y_A_test = train_test_split(X_Agora, Y_Agora, test_size=0.2, random_state=0) 
SVC_Agora = LinearSVC().fit(X_A_train, Y_A_train)

print('Training for WebIQ')
X_W_train, X_W_test, Y_W_train, Y_W_test = train_test_split(X_WebIQ, Y_WebIQ, test_size=0.2, random_state=0) 
SVC_WebIQ = LinearSVC().fit(X_W_train, Y_W_train)

Training for Agora
Training for WebIQ


In [39]:
Y_P_Agora = SVC_Agora.predict(X_A_test)
precision_recall_fscore_support(Y_P_Agora, Y_A_test, average='micro')

(0.9614741125932625, 0.9614741125932625, 0.9614741125932625, None)

In [40]:
Y_P_WebIQ = SVC_WebIQ.predict(X_W_test)
precision_recall_fscore_support(Y_P_WebIQ, Y_W_test, average='micro')

(0.8373597929249353, 0.8373597929249353, 0.8373597929249353, None)